In [1]:
import numpy as np
import feature_finder_f as ff
from datetime import datetime
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.metrics import mutual_info_score
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
import numpy.ma as ma

Could not import rpy package
Could not import r-package RCIT


In [2]:
def calc_MI(x, y, bins):
    c_xy = np.histogram2d(x, y, bins)[0]
    g, p, dof, expected = chi2_contingency(c_xy, lambda_="log-likelihood")
    mi = 0.5 * g / c_xy.sum()
    return(mi, p)

In [3]:
def crossMI(datax, datay, lag=1, bins = 10):   
    return(calc_MI(datax[lag:], datay[:-lag],bins))

In [7]:
def calc_MI(x, y, bins = 10):
    c_xy = np.histogram2d(x, y, bins)[0]
    g, p, dof, expected = chi2_contingency(c_xy, lambda_="log-likelihood")
    #mi = 0.5 * g / c_xy.sum()
    mi = mutual_info_score(None, None, contingency=c_xy)
    return(mi, p)

def cross_MI(datax, datay, lag=1, bins = 10):
    return(calc_MI(datax[lag:], datay[:-lag],bins))


In [39]:
tau = 1
tau_min = tau
tau_max = tau+11
level = 0.05

In [42]:
    result_extremes = np.array(count)
    result_extremes = result_extremes.reshape((-1,1))
    componenet = []
    lag = []
    result_sst = np.array(ts)

In [47]:
test_type = "all"

In [51]:
i = 20

In [56]:
sst = result_sst[:,i].reshape((-1,1))
data = np.concatenate((result_extremes,sst), axis=1)
data = np.array(data)
df = pd.DataFrame(data)

In [60]:
lag_range = range(tau_min, tau_max+1)
r = grangercausalitytests(df,maxlag=lag_range,  verbose=False)

In [66]:
        p = np.zeros(4)
        for j in lag_range:
            p[0] = r[j][0]['lrtest'][1]
            p[1] = r[j][0]['params_ftest'][1]
            p[2] = r[j][0]['ssr_chi2test'][1]
            p[3] = r[j][0]['ssr_ftest'][1]


            if test_type == "all" and np.all(p < level):
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'lrtest' and p[0] < level:
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'params_ftest' and p[1] < level:
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'ssr_chi2test' and p[2] < level:
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'ssr_ftest' and p[3] < level:
                componenet.append(i+1)
                lag.append(-j)

In [68]:
r

{1: ({'lrtest': (0.1932365712605133, 0.6602363891688867, 1),
   'params_ftest': (0.19189726253893621, 0.6615695830964576, 416.0, 1.0),
   'ssr_chi2test': (0.19328113702841745, 0.6601996714126976, 1),
   'ssr_ftest': (0.1918972625389538, 0.6615695830964576, 416.0, 1)},
   array([[0., 1., 0.]])]),
 2: ({'lrtest': (1.2583647701908376, 0.5330274340672316, 2),
   'params_ftest': (0.6225929498028248, 0.5370541884303011, 413.0, 2.0),
   'ssr_chi2test': (1.2602607894312796, 0.5325223583780274, 2),
   'ssr_ftest': (0.6225929498027732, 0.5370541884303381, 413.0, 2)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'lrtest': (2.1057851819696225, 0.550743268568918, 3),
   'params_ftest': (0.6918909238353729, 0.5574029749224949, 410.0, 3.0),
   'ssr_chi2test': (2.111111087117207, 0.549668187443767, 3),
   'ssr_ftest': (0.6918909238353755, 0.5574029749224949, 410.0, 3)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0

In [140]:
def granger_generator(ts, count, data_sst, V, test_type = "all", tau_min = 1, tau_max = 12,level = 0.05):
    result_extremes = np.array(count)
    result_extremes = result_extremes.reshape((-1,1))
    componenet = []
    lag = []
    result_sst = np.array(ts)

    for i in range(result_sst.shape[1]):
        sst = result_sst[:,i].reshape((-1,1))
        data = np.concatenate((result_extremes,sst), axis=1)
        data = np.array(data)
        df = pd.DataFrame(data)
        lag_range = range(tau_min, tau_max+1)
        r = grangercausalitytests(df,maxlag=lag_range,  verbose=False)
        p = np.zeros(4)
        for j in lag_range:
            p[0] = r[j][0]['lrtest'][1]
            p[1] = r[j][0]['params_ftest'][1]
            p[2] = r[j][0]['ssr_chi2test'][1]
            p[3] = r[j][0]['ssr_ftest'][1]


            if test_type == "all" and np.all(p < level):
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'lrtest' and p[0] < level:
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'params_ftest' and p[1] < level:
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'ssr_chi2test' and p[2] < level:
                componenet.append(i+1)
                lag.append(-j)
            elif test_type == 'ssr_ftest' and p[3] < level:
                componenet.append(i+1)
                lag.append(-j)

    link = np.array(list(zip((componenet),(lag))))
    
    df = ff.data_list_maker_V(data_sst, V, link)
    deleted_index = []
    componenets = set(link[:,0])
    for componenet in componenets:
        componenet_index = (link[:,0] == componenet)
        componenet_list = link[componenet_index]
        Index = componenet_index.nonzero()[0]
        sorted_index = np.argsort(componenet_list[:,1],axis=0)
        mx = ma.masked_array(componenet_list)
        for i in range(len(sorted_index)):
            if ma.is_masked(mx[sorted_index[i]]): continue
            for j in range(i+1,len(sorted_index)):
                if (not ma.is_masked(mx[sorted_index[i]]) and (df.iloc[:,Index[sorted_index[i]]].corr(df.iloc[:,Index[sorted_index[j]]]) > 0.7)):
                    mx[sorted_index[j]] = ma.masked

        if not np.isscalar(mx.mask):
            deleted_index.extend(Index[mx.mask[:,0].nonzero()[0]])
    deleted_index = np.array(deleted_index)
    link = np.delete(link,deleted_index,axis=0) 
    return(link)


In [5]:
link = ff.granger_generator(ts, count, data_sst, V, auto_corr=0.7)

In [6]:
link

array([[  3, -10],
       [ 15,  -5],
       [ 27,  -1],
       [ 29,  -6],
       [ 29,  -8],
       [ 29, -11],
       [ 30,  -2],
       [ 30,  -4],
       [ 30,  -6],
       [ 30,  -8],
       [ 30, -10],
       [ 30, -12],
       [ 35,  -2],
       [ 40,  -4],
       [ 41,  -2],
       [ 41,  -4]])

In [84]:
df = ff.data_list_maker_V(data_sst, V, link)
deleted_index = []
componenets = set(link[:,0])

In [144]:
link

array([[  3, -10],
       [ 15,  -5],
       [ 27,  -1],
       [ 29,  -6],
       [ 29,  -8],
       [ 29, -11],
       [ 30,  -2],
       [ 30,  -4],
       [ 30,  -6],
       [ 30,  -8],
       [ 30, -10],
       [ 30, -12],
       [ 35,  -2],
       [ 40,  -4],
       [ 41,  -2],
       [ 41,  -4]])

In [141]:
link

array([[  3,  -9],
       [  3, -10],
       [ 15,  -5],
       [ 27,  -1],
       [ 29,  -5],
       [ 29,  -6],
       [ 29,  -7],
       [ 29,  -8],
       [ 29, -10],
       [ 29, -11],
       [ 30,  -1],
       [ 30,  -2],
       [ 30,  -3],
       [ 30,  -4],
       [ 30,  -5],
       [ 30,  -6],
       [ 30,  -7],
       [ 30,  -8],
       [ 30,  -9],
       [ 30, -10],
       [ 30, -11],
       [ 30, -12],
       [ 35,  -2],
       [ 40,  -4],
       [ 41,  -1],
       [ 41,  -2],
       [ 41,  -3],
       [ 41,  -4]])

In [85]:
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
12,-15.658477,-18.875230,-1.462725,-2.494005,-4.742317,-11.971237,-8.811559,-5.876422,-9.537527,-8.610736,...,-6.870880,-10.414479,-12.280805,-22.539744,-3.862890,-12.410080,2.374501,9.848307,3.059157,-7.286640
13,-16.247750,-15.658477,-7.388692,5.868590,-3.620935,-4.742317,-11.971237,-8.811559,-4.718250,-9.537527,...,-13.103352,-6.870880,-10.414479,-12.280805,-3.827599,-9.823005,10.403320,2.374501,9.848307,3.059157
14,-8.978083,-16.247750,4.445281,-3.602385,-1.406557,-3.620935,-4.742317,-11.971237,-5.876422,-4.718250,...,-11.078641,-13.103352,-6.870880,-10.414479,-6.044788,-15.887835,2.017544,10.403320,2.374501,9.848307
15,-7.691997,-8.978083,16.317852,-18.548149,7.062464,-1.406557,-3.620935,-4.742317,-8.811559,-5.876422,...,-23.622484,-11.078641,-13.103352,-6.870880,-7.999546,-5.584725,8.935667,2.017544,10.403320,2.374501
16,-3.818863,-7.691997,12.782455,-18.941576,0.416635,7.062464,-1.406557,-3.620935,-11.971237,-8.811559,...,-9.145543,-23.622484,-11.078641,-13.103352,-7.031180,-8.985496,13.214939,8.935667,2.017544,10.403320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,24.076212,3.987968,9.603717,14.449409,13.479496,15.934460,2.721533,-0.570186,-0.030342,-2.645194,...,-5.008418,-2.881616,-0.178987,3.182010,-12.331287,-11.242183,18.162507,15.190981,18.763838,19.803094
416,29.419209,24.076212,15.877617,7.526736,17.377247,13.479496,15.934460,2.721533,-3.654015,-0.030342,...,-4.186917,-5.008418,-2.881616,-0.178987,-0.051098,-0.189972,11.615687,18.162507,15.190981,18.763838
417,23.974378,29.419209,28.776422,8.523444,16.446438,17.377247,13.479496,15.934460,-0.570186,-3.654015,...,-6.329120,-4.186917,-5.008418,-2.881616,3.454851,17.447818,5.580168,11.615687,18.162507,15.190981
418,22.348548,23.974378,28.537041,8.597736,12.915137,16.446438,17.377247,13.479496,2.721533,-0.570186,...,5.933121,-6.329120,-4.186917,-5.008418,10.450159,10.315818,6.520422,5.580168,11.615687,18.162507


In [95]:
link

array([[  3,  -9],
       [  3, -10],
       [ 15,  -5],
       [ 27,  -1],
       [ 29,  -5],
       [ 29,  -6],
       [ 29,  -7],
       [ 29,  -8],
       [ 29, -10],
       [ 29, -11],
       [ 30,  -1],
       [ 30,  -2],
       [ 30,  -3],
       [ 30,  -4],
       [ 30,  -5],
       [ 30,  -6],
       [ 30,  -7],
       [ 30,  -8],
       [ 30,  -9],
       [ 30, -10],
       [ 30, -11],
       [ 30, -12],
       [ 35,  -2],
       [ 40,  -4],
       [ 41,  -1],
       [ 41,  -2],
       [ 41,  -3],
       [ 41,  -4]])

In [121]:
componenet = 41

In [122]:
componenet_index = (link[:,0] == componenet)

In [123]:
componenet_index

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True])

In [124]:
componenet_list = link[componenet_index]

In [125]:
componenet_list

array([[41, -1],
       [41, -2],
       [41, -3],
       [41, -4]])

In [126]:
Index = componenet_index.nonzero()[0]

In [127]:
Index

array([24, 25, 26, 27])

In [128]:
sorted_index = np.argsort(componenet_list[:,1],axis=0)

In [129]:
sorted_index

array([3, 2, 1, 0])

In [130]:
componenet_list

array([[41, -1],
       [41, -2],
       [41, -3],
       [41, -4]])

In [131]:
mx = ma.masked_array(componenet_list)
for i in range(len(sorted_index)):
    if ma.is_masked(mx[sorted_index[i]]): continue
    for j in range(i+1,len(sorted_index)):
        if (not ma.is_masked(mx[sorted_index[i]]) and (df.iloc[:,Index[sorted_index[i]]].corr(df.iloc[:,Index[sorted_index[j]]]) > 0.8)):
            mx[sorted_index[j]] = ma.masked

In [132]:
mx

masked_array(
  data=[[41, -1],
        [41, -2],
        [41, -3],
        [41, -4]],
  mask=False,
  fill_value=999999)

In [103]:
sorted_index

array([11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [133]:
i=0

In [106]:
Index[sorted_index[i]]

21

In [108]:
link[21]

array([ 30, -12])

In [107]:
df.iloc[:,Index[sorted_index[i]]]

12    -22.539744
13    -12.280805
14    -10.414479
15     -6.870880
16    -13.103352
         ...    
415     3.182010
416    -0.178987
417    -2.881616
418    -5.008418
419    -4.186917
Name: 21, Length: 408, dtype: float64

In [138]:
j=i+3

In [139]:
df.iloc[:,Index[sorted_index[i]]].corr(df.iloc[:,Index[sorted_index[j]]])

0.4652663873421293

In [81]:

    for componenet in componenets:
        componenet_index = (link[:,0] == componenet)
        componenet_list = link[componenet_index]
        Index = componenet_index.nonzero()[0]
        sorted_index = np.argsort(componenet_list[:,1],axis=0)
        mx = ma.masked_array(componenet_list)
        for i in range(len(sorted_index)):
            if ma.is_masked(mx[sorted_index[i]]): continue
            for j in range(i+1,len(sorted_index)):
                if (not ma.is_masked(mx[sorted_index[i]]) and (df.iloc[:,Index[sorted_index[i]]].corr(df.iloc[:,Index[sorted_index[j]]]) > 0.8)):
                    mx[sorted_index[j]] = ma.masked

        if not np.isscalar(mx.mask):
            deleted_index.extend(Index[mx.mask[:,0].nonzero()[0]])
    deleted_index = np.array(deleted_index)
    link = np.delete(link,deleted_index,axis=0) 

array([[  3, -10],
       [ 15,  -5],
       [ 27,  -1],
       [ 29,  -5],
       [ 29,  -6],
       [ 29,  -7],
       [ 29,  -8],
       [ 29, -10],
       [ 29, -11],
       [ 30,  -1],
       [ 30,  -2],
       [ 30,  -3],
       [ 30,  -4],
       [ 30,  -5],
       [ 30,  -6],
       [ 30,  -7],
       [ 30,  -8],
       [ 30,  -9],
       [ 30, -10],
       [ 30, -11],
       [ 30, -12],
       [ 35,  -2],
       [ 40,  -4],
       [ 41,  -1],
       [ 41,  -2],
       [ 41,  -3],
       [ 41,  -4]])

In [2]:
train_start = 1976
validation_end = 2010
f = 12
cc = 'ET'
n_components_sst = 57

In [3]:
temporal_limits = {"time_min":datetime(train_start, 1, 1, 0, 0),"time_max":datetime(validation_end, 12, 1, 0, 0)}
original_count, count = ff.drought_timeseries("./{}_gamma_18912015_{}.npy".format(cc,f),train_start,validation_end)

In [4]:
data_sst, ts, V, df_sst, avg, std = ff.PCA_computer_rotated('../../../nc/sst.mnmean.nc', "sst",temporal_limits, n_components_sst, -9.96921e+36)

In [94]:
y = original_count

In [95]:
x = ts[:,20]

In [101]:
r = []
for tau in np.arange(1,13):
    r.append(crossMI(x,y,tau,10))

In [102]:
r

[(0.13635516760515987, 0.008807370799585976),
 (0.11373617229994125, 0.1356944078491482),
 (0.11326945485394589, 0.1454308662194523),
 (0.11281034567135093, 0.15553195976067463),
 (0.12508643484417614, 0.04464739236655455),
 (0.13420765543975544, 0.014811560636947446),
 (0.12838487597978465, 0.032420658209457484),
 (0.14814173310785192, 0.0021730763708920522),
 (0.11061378660206903, 0.21130759935119353),
 (0.11829930284519549, 0.10847455918816165),
 (0.13099446204800289, 0.02749775431408723),
 (0.13013874747098222, 0.03172464785623767)]

In [96]:
calc_MI(x,y,10)

(0.13821540923000122, 0.006423903980565155)

In [74]:
    c_xy = np.histogram2d(x, y, 10)[0]
    g, p, dof, expected = chi2_contingency(c_xy, lambda_="log-likelihood")

In [83]:
result = []
for i in range(ts.shape[1]):
    x = ts[:,i]
    #result.append(calc_MI(x,y,10)[1])
    result.append(stats.pearsonr(x,y)[1])

In [84]:
result = np.array(result)

In [85]:
b = np.where(result < 0.01)

In [48]:
np.count_nonzero(result < 0.05)

21

In [57]:
a[0]

array([ 2,  3,  4,  5,  8,  9, 13, 16, 17, 20, 25, 32, 33, 34, 39, 42, 45,
       46, 48, 49, 50])

In [58]:
b[0]

array([ 0,  1,  2,  4,  5,  8,  9, 10, 12, 16, 19, 20, 21, 25, 27, 28, 30,
       32, 33, 37, 49, 56])

In [86]:
np.intersect1d(a[0],b[0])


array([ 2,  5,  8, 20, 33, 49])